In [1]:
%load_ext autoreload
%autoreload 2

import os; import sys; from pathlib import Path
root = Path(os.getcwd()).parents[0]
sys.path.append(str(root))


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import numpy as np
from src.data import post_process


In [3]:
# load raw data

df = pd.read_csv('../data/booli_1000_pages_raw_20210506.csv')

display(
    df.head(3),
    f'shape = {df.shape}'
)


,Slutpris,Utropspris,Prisutveckling,Såld eller borttagen,Slutpris/m²,Dagar på Booli,Avgift,Bostadstyp,Driftskostnad,Våning,Byggår,BRF,adress,area_and_n_rooms,location,url,Energiklass
0,5 950 000 kr,4 690 000 kr,+1 260 000 kr (+26.9 %),2021-05-05,108 200 kr/m²,13,3 178 kr/mån,Lägenhet,400 kr/mån,5 tr,1904.0,BRF Bergskyrkan,Bergs­gatan 59,"55 m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4201785,NaN
1,8 000 000 kr,6 695 000 kr,+1 305 000 kr (+19.5 %),2021-05-06,115 900 kr/m²,6,3 876 kr/mån,Lägenhet,599 kr/mån,2 tr,2019.0,BRF Tegeludden 16,Östhammars­gatan 70B,"69 m², 3 rum","Lägenhet, Gärdet",https://www.booli.se/bostad/4099014,NaN
2,6 500 000 kr,4 950 000 kr,+1 550 000 kr (+31.3 %),2021-05-05,97 700 kr/m²,12,5 041 kr/mån,Lägenhet,250 kr/mån,4 tr,1997.0,BRF St.Erik 1,Carl-Gustaf Lindstedts gata 10,"66½ m², 2 rum","Lägenhet, Kungsholmen",https://www.booli.se/annons/4201484,NaN


'shape = (12658, 17)'

In [4]:
# post process data

df_pp = post_process(df)

display(
    df_pp.head(3),
    f'shape = {df_pp.shape}'
)


/Users/sebastian/miniconda3/envs/housing_price_prediction/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,price_ask,price_gain,sell_date,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
0,1995000.0,12.8,2020-05-07,935.0,280.0,1.0,1929.0,18.0,1.0,110833.333333,0,0,0,0,0,1
1,3595000.0,-2.6,2020-05-07,1859.0,550.0,2.0,1974.0,40.0,2.0,89875.000000,1,0,0,0,0,0
2,4695000.0,5.4,2020-05-07,2334.0,NaN,4.0,1916.0,53.0,2.0,88584.905660,0,0,0,0,0,1


'shape = (12658, 16)'

In [5]:
# split covariates & target

#X = df_pp.drop('sell_date', axis=1).dropna() 
X = df_pp.drop('sell_date', axis=1).fillna(-1)
y = X[['price_gain']]
X = X.drop(columns=['price_gain'])

#X.drop(index=y[y.price_gain==0].index, inplace=True)
#y.drop(index=y[y.price_gain==0].index, inplace=True)

display(
    X.head(3), f'shape = {X.shape}',
    y.head(3), f'shape = {y.shape}',
)


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
0,1995000.0,935.0,280.0,1.0,1929.0,18.0,1.0,110833.333333,0,0,0,0,0,1
1,3595000.0,1859.0,550.0,2.0,1974.0,40.0,2.0,89875.000000,1,0,0,0,0,0
2,4695000.0,2334.0,-1.0,4.0,1916.0,53.0,2.0,88584.905660,0,0,0,0,0,1


'shape = (12658, 14)'

,price_gain
0,12.8
1,-2.6
2,5.4


'shape = (12658, 1)'

In [6]:
# split data
# sell order old --> recent, keep order by not shuffel and test on the most recent ones

train_ratio = 0.8; val_ratio = 0.1; test_ratio = 0.1
assert train_ratio + val_ratio + test_ratio == 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + val_ratio), shuffle=False) 

display(
    X_train.iloc[[0, -1]], 
    f'train shape X: {X_train.shape} y: {y_train.shape} ratio: {X_train.shape[0]/ X.shape[0]:.2f}',
    X_val.iloc[[0, -1]],
    f'val shape X: {X_val.shape} y: {y_val.shape} ratio: {X_val.shape[0]/ X.shape[0]:.2f}',
    X_test.iloc[[0, -1]],
    f'test shape X: {X_test.shape} y: {y_test.shape} ratio: {X_test.shape[0]/ X.shape[0]:.2f}'
)


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
0,1995000.0,935.0,280.0,1.0,1929.0,18.0,1.0,110833.333333,0,0,0,0,0,1
10125,4295000.0,35803.0,417.0,1.0,1937.0,59.0,2.0,72796.610169,0,0,0,0,0,1


'train shape X: (10126, 14) y: (10126, 1) ratio: 0.80'

,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
10126,5450000.0,2581.0,-1.0,1.0,1930.0,55.0,2.0,99090.909091,0,0,0,1,0,0
11391,2950000.0,2192.0,294.0,2.0,1926.0,37.0,1.5,79729.729730,1,0,0,0,0,0


'val shape X: (1266, 14) y: (1266, 1) ratio: 0.10'

,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
11392,5795000.0,5200.0,441.0,5.0,2000.0,81.5,3.5,71104.294479,0,0,0,1,0,0
12657,4895000.0,2915.0,-1.0,6.0,1981.0,67.0,2.0,73059.701493,0,0,1,0,0,0


'test shape X: (1266, 14) y: (1266, 1) ratio: 0.10'

In [7]:
# standardize numerical columns

num_cols, cat_cols = X.columns[:8], X.columns[8:]

scaler = preprocessing.StandardScaler()
scaler.fit(X_train[num_cols])
joblib.dump(scaler, '../data/scaler.pkl') # save scaler for future inference

X_train = pd.concat([
    pd.DataFrame(
        data=scaler.transform(X_train[num_cols]),
        columns=num_cols,
        index=X_train.index
    ),
    X_train[cat_cols]
    ],
    axis=1
)

X_val = pd.concat([
    pd.DataFrame(
        data=scaler.transform(X_val[num_cols]),
        columns=num_cols,
        index=X_val.index
    ),
    X_val[cat_cols]
    ],
    axis=1
)

X_test = pd.concat([
    pd.DataFrame(
        data=scaler.transform(X_test[num_cols]),
        columns=num_cols,
        index=X_test.index
    ),
    X_test[cat_cols]
    ],
    axis=1
)

display(
    X_train.iloc[[0, -1]],
    X_val.iloc[[0, -1]],
    X_test.iloc[[0, -1]],
)


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
0,-0.866244,-1.316443,-0.16946,-0.759087,0.310326,-1.377816,-1.272198,0.951883,0,0,0,0,0,1
10125,-0.207233,24.388671,0.24132,-0.759087,0.324520,-0.016174,-0.290542,-0.414131,0,0,0,0,0,1


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
10126,0.123705,-0.102992,-1.012009,-0.759087,0.312100,-0.149017,-0.290542,0.530177,0,0,0,1,0,0
11391,-0.592611,-0.389768,-0.127482,-0.337550,0.305003,-0.746811,-0.781370,-0.165142,1,0,0,0,0,0


,price_ask,montly_fee_cost,montly_operating_cost,floor,construction_year,area,n_rooms,list_price_per_m2,location_kungsholmen,location_gärdet,location_södermalm,location_other,location_östermalm,location_vasastan
11392,0.222557,1.827766,0.313282,0.927061,0.436299,0.731068,1.181943,-0.474908,0,0,0,1,0,0
12657,-0.035317,0.143237,-1.012009,1.348598,0.402588,0.249512,-0.290542,-0.404683,0,0,1,0,0,0


In [8]:
# save ml data

data = [X_train, X_val, X_test, y_train, y_val, y_test]
names = ['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test']

for d, n in zip(data, names):
    d.to_csv(f'../data/{n}.csv', index=False)
    